In [2]:
import sys
import os
sys.path.append(os.path.abspath('../')) ## needed to import the function.py file
from functions import *
import pandas as pd

## plot settings
color_discrete_sequence = color_discrete_kuzu #imported from globalvars.py
template = template ##imported from globalvars.py

2023-05-26 08:51:49.204697: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# specify the path of the Excel file
file_path = '../../exports/tables/Themencluster_Fachexperten_BERTTopic_paraphrase-multilingual-MiniLM-L12-v2_Themencluster_Fachexperten.xlsx'
sheet_name = 'Cluster_L2'  # sheet name

# import the Excel sheet as a pandas DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)
df_export = pd.read_excel(file_path, sheet_name='export')

In [4]:
## Import docs
filelocation = '../../data/DataTextTrain'
data = pd.read_feather(filelocation)
data = data
docs = data["Kommentar"].values
timestamps = data.yearquarter.to_list()

## Modelvariante 1: Merged Topic Model

In [5]:
# Load the Original Topic Model
topic_model = BERTopic.load("../../models/BERTTopic_paraphrase-multilingual-MiniLM-L12-v2.model")

In [6]:
df_print = df[df['Themencluster']!="Outlier"].sort_values(by='Anzahl Kommentare',ascending=False) # Filter out Sonstige

In [7]:
df_print["Anzahl %"]=(df_print["Anzahl Kommentare"]/sum(df_print["Anzahl Kommentare"])).round(3)

In [8]:
df_print

,Themencluster,Anzahl Kommentare,Anzahl %
1,Pünktlichkeit,4224,0.157
2,Platzangebot Zug,2922,0.108
3,Preis-Leistung,2400,0.089
4,WC Verfügbarkeit und Sauberkeit,1785,0.066
5,Vertrieb und Services,1663,0.062
6,Fahrplanangebot,1655,0.061
7,Sonstiges,1642,0.061
8,Corona,1579,0.059
9,Lob,1435,0.053
10,"Platzangebot Velo, Gepäck, Kinderwagen",1408,0.052


In [9]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(df_print.sort_values('Anzahl Kommentare', ascending=True),
    y='Themencluster', 
    x='Anzahl %',
    opacity=1,
    color_discrete_sequence=color_discrete_sequence
)

fig.update_layout(
    title="Die 18 finalen Themencluster: Relative Häufigkeiten",
    xaxis_title='',
    yaxis_title='',
    xaxis=dict(
        tickangle=0
    ),
    width=900, 
    height=450,
    legend_title=""
)
fig.update_xaxes(range=(0.001, 0.2), title=dict(text='Relative Kommentarhäufigkeit', font=dict(size=12)))
fig.update_yaxes(title=dict(text='Themencluster', font=dict(size=12)))
fig.show()

pio.write_image(fig, "../../exports/images/fig-18-Themencluster-expertenreview.svg") #save fig as svg

In [28]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(df_print.sort_values('Anzahl Kommentare', ascending=False),
    x='Themencluster', 
    y='Anzahl %',
    opacity=1,
    color_discrete_sequence=color_discrete_sequence,
    orientation='v',
    text=df_print['Anzahl %'].apply(lambda x: f"{x*100:.0f}%")  # Format labels as percentages with one decimal place
)

fig.update_traces(textposition='auto', textangle=0,textfont=dict(size=10))  # Keep labels horizontal

fig.update_layout(
    title="Die 18 finalen Themencluster: Relative Häufigkeiten",
    xaxis_title='',
    yaxis_title='',
    xaxis=dict(
        tickangle=45
    ),
    width=900, 
    height=600,
    legend_title=""
)
fig.update_yaxes(range=(0.001, 0.18), title=dict(text='Relative Häufigkeit', font=dict(size=12)))
#fig.update_yaxes(title=dict(text='Themencluster', font=dict(size=12)))
fig.show()

pio.write_image(fig, "../../exports/images/fig-18-Themencluster-expertenreview.svg")  # Save fig as SVG




In [22]:
# generate list for topic merge for new aggregated model
grouped = df_export.groupby('cluster_level_2')['Topic'].agg(list).reset_index()
topics_merge_list = grouped.Topic.tolist()

In [23]:
# Merge Topics based on clusters
topic_model.merge_topics(docs, topics_merge_list)

In [24]:
# Assign Custom Labels based on Clustering to the Topics
df_new_model = topic_model.get_topic_info()
df_new_lables = df_new_model.merge(df,left_on='Count',right_on='Anzahl Kommentare') # Joinin df to have new Topic ids and Custom cluster namens
df_new_lables.set_index('Topic', inplace=True) # Set the 'Topic' column as the index of the DataFrame
id_mapping_dict = df_new_lables['Themencluster'].to_dict() # Create a dictionary using the index and 'Themencluster' column
topic_model.set_topic_labels(id_mapping_dict) # Assign Lables to model
df_export = topic_model.get_topic_info()

In [25]:
# Save the Model
topic_model.save('../../models/BERTTopic_paraphrase-multilingual-MiniLM-L12-v2_merged.model')
df_export.to_excel('../../exports/tables/merged_topics.xlsx', index=False)

In [26]:
df_export

,Topic,Count,Name,CustomName
0,-1,18072,-1_zug_züge_klasse_app,Outlier
1,0,4224,0_zug_verspätung_minuten_verspätungen,Pünktlichkeit
2,1,2922,1_klasse_wagen_sitzplätze_stosszeiten,Platzangebot Zug
3,2,2400,2_teuer_preise_ga_preis,Preis-Leistung
4,3,1785,3_wc_toiletten_wcs_toilette,WC Verfügbarkeit und Sauberkeit
5,4,1663,4_app_ticket_lösen_mobile,Vertrieb und Services
6,5,1655,5_postauto_anschluss_minuten_busse,Fahrplanangebot
7,6,1642,6_hund_kinderwagen_perron_hunde,Sonstiges
8,7,1579,7_maskenpflicht_maske_masken_tragen,Corona
9,8,1435,8_zufrieden_super_personal_job,Lob


## Modelvariante 2: Merged Topic Model with Outlier Reduction

In [27]:
# # Save outliers reduced model for evaluation comparison
# topics, probs = topic_model.transform(docs)
# new_topics = topic_model.reduce_outliers(docs, topics) # Reduce outliers
# topic_model.update_topics(docs, topics=new_topics) # update Model
# topic_model.save('../../models/BERTTopic_paraphrase-multilingual-MiniLM-L12-v2_merged_oulier_reduced.model') # Save Model

In [28]:
# Load the Outlier reduced Model
topic_model_outlier_reduced = BERTopic.load("../../models/BERTTopic_paraphrase-multilingual-MiniLM-L12-v2_merged_oulier_reduced.model")

In [36]:
df_2 = topic_model_outlier_reduced.get_topic_info()
df_print_2 = df_2[df_2['CustomName']!="Outlier"].sort_values(by='Count',ascending=False) # Filter out Sonstige

In [39]:
df_print_2

,Topic,Count,Name,CustomName,Anzahl %
1,0,8341,0_zug_der_nach_die,Pünktlichkeit,0.187
2,1,5123,1_klasse_mehr_der_die,Platzangebot Zug,0.115
3,2,4106,2_preise_teuer_für_ich,Preis-Leistung,0.092
5,4,3474,4_app_ich_nicht_das,Vertrieb und Services,0.078
6,5,3024,5_bus_der_nach_in,Fahrplanangebot,0.068
9,8,2568,8_sbb_zufrieden_ich_sehr,Lob,0.057
10,9,2235,9_für_gepäck_platz_koffer,"Platzangebot Velo, Gepäck, Kinderwagen",0.050
7,6,2183,6_und_die_mit_auf,Sonstiges,0.049
4,3,1950,3_wc_toiletten_die_und,WC Verfügbarkeit und Sauberkeit,0.044
8,7,1844,7_maskenpflicht_maske_die_tragen,Corona,0.041


In [42]:
df_print_2["Anzahl %"]=(df_print_2["Count"]/sum(df_print_2["Count"])).round(3)

In [44]:
import plotly.express as px
import plotly.io as pio

fig = px.bar(df_print_2.sort_values('Count', ascending=True),
    y='CustomName', 
    x='Anzahl %',
    opacity=1,
    color_discrete_sequence=color_discrete_sequence
)

fig.update_layout(
    title="Die 18 finalen Themencluster: Relative Häufigkeiten",
    xaxis_title='',
    yaxis_title='',
    xaxis=dict(
        tickangle=0
    ),
    width=900, 
    height=450,
    legend_title=""
)
fig.update_xaxes(range=(0.001, 0.2), title=dict(text='Relative Kommentarhäufigkeit', font=dict(size=12)))
fig.update_yaxes(title=dict(text='Themencluster', font=dict(size=12)))
fig.show()

pio.write_image(fig, "../../exports/images/fig-18-Themencluster-expertenreview_reduced.svg") #save fig as svg